# Qwen2 SFT 教学代码（LoRA 版）

本 Notebook 参考 `Qwen-Pre-Post-train/SFT.ipynb` 的思路，整理成一份更适合教学/复现的最小可跑 SFT 示例：

- 训练数据采用对话格式（system/user/assistant）
- 只对 assistant 内容计算 loss（其他 token 的 label 置为 `-100`）
- 使用 LoRA 做参数高效微调（更省显存、更适合教学）


## 0. 环境与准备

- 环境：`conda activate llm`（或在 Jupyter 里选择 `llm` 内核）
- 模型：`qwen/Qwen2-0.5B-Instruct`
- 离线加载：建议提前把模型下载到 `MODELSCOPE_CACHE`，否则首次运行会尝试联网下载。


In [1]:
import os
import random
import sys
from dataclasses import dataclass
from pathlib import Path
from typing import Any, Dict, List

import torch
from torch.utils.data import DataLoader

print("python:", sys.executable)
print("torch:", torch.__version__, "cuda:", torch.cuda.is_available())

# 指定 modelscope 缓存目录（按需修改）
os.environ.setdefault("MODELSCOPE_CACHE", r"D:/myProject/modelscope_hub")
print("MODELSCOPE_CACHE:", os.environ["MODELSCOPE_CACHE"])

# 固定随机种子（方便复现）
seed = 42
random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

device = "cuda" if torch.cuda.is_available() else "cpu"
device


python: e:\Softwares\anaconda3\envs\llm\python.exe
torch: 2.10.0+cu126 cuda: True
MODELSCOPE_CACHE: D:/myProject/modelscope_hub


'cuda'

In [2]:
from datasets import Dataset
from modelscope import AutoModelForCausalLM, AutoTokenizer

# 你可以填 ModelScope 的模型 ID；如果本地已缓存，也可以填本地目录。
model_name_or_path = "qwen/Qwen2-0.5B-Instruct"

# 这里根据你当前工程的缓存结构做一个“优先本地目录”的兜底（可选）
local_dir = Path(os.environ["MODELSCOPE_CACHE"]) / "models" / "qwen" / "Qwen2-0___5B-Instruct"
if local_dir.exists():
    model_name_or_path = str(local_dir)
print("model_name_or_path:", model_name_or_path)

# 根据硬件选择 dtype
if device == "cuda":
    dtype = torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16
else:
    dtype = torch.float32
print("dtype:", dtype)

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
if tokenizer.pad_token_id is None:
    # Qwen2 通常用 <|endoftext|> 做 pad
    tokenizer.pad_token = "<|endoftext|>"
tokenizer.padding_side = "right"

model = AutoModelForCausalLM.from_pretrained(model_name_or_path, torch_dtype=dtype)
model.to(device)
model.config.use_cache = False
model.config.pad_token_id = tokenizer.pad_token_id

print("pad_token_id:", tokenizer.pad_token_id, "eos_token_id:", tokenizer.eos_token_id)


model_name_or_path: D:\myProject\modelscope_hub\models\qwen\Qwen2-0___5B-Instruct
dtype: torch.bfloat16


`torch_dtype` is deprecated! Use `dtype` instead!


pad_token_id: 151643 eos_token_id: 151645


In [5]:
SYSTEM_PROMPT = "You are a helpful assistant."

@torch.inference_mode()
def chat(m, tok, prompt: str, max_new_tokens: int = 128) -> str:
    m.eval()
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": prompt},
    ]
    text = tok.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tok(text, return_tensors="pt").to(device)
    outputs = m.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=False,
        eos_token_id=tok.eos_token_id,
        pad_token_id=tok.pad_token_id,
        temperature=0
    )
    gen_ids = outputs[0, inputs["input_ids"].shape[1] :]
    return tok.decode(gen_ids, skip_special_tokens=True)

print("[Before SFT] 2+2:", chat(model, tokenizer, "2+2等于几？"))
print("[Before SFT] 你是谁:", chat(model, tokenizer, "你是谁？"))
print("[Before SFT] 用一句话解释什么是SFT。:", chat(model, tokenizer, "用一句话解释什么是SFT。"))
print("[Before SFT] 写一个 Python 函数，计算两个数的和。:", chat(model, tokenizer, "写一个 Python 函数，计算两个数的和。"))
print("[Before SFT] 给我一个 3 条的番茄工作法要点列表。:", chat(model, tokenizer, "给我一个 3 条的番茄工作法要点列表。"))
print("[Before SFT] 把“我喜欢机器学习”翻译成英文。:", chat(model, tokenizer, "把“我喜欢机器学习”翻译成英文。"))



[Before SFT] 2+2: 2 + 2 等于 4。
[Before SFT] 你是谁: 我是来自阿里云的大规模语言模型，我叫通义千问。
[Before SFT] 用一句话解释什么是SFT。: SFT是美国联邦政府的财政审计机构，负责对联邦政府的财务状况进行审查和监督。
[Before SFT] 写一个 Python 函数，计算两个数的和。: 这是一个简单的Python函数，它可以接受两个数字作为输入，并返回它们的和：

```python
def add_numbers(num1, num2):
    return num1 + num2

# 示例用法
result = add_numbers(3, 5)
print(result)  # 输出: 8
```

在这个函数中，我们定义了一个名为`add_numbers`的函数，它接受两个参数`num1`和`num2`。然后，我们使用加号运算符将这两个数字相加，并返回结果。

在示例用法中，我们调用了这个函数并传入了三个数字
[Before SFT] 给我一个 3 条的番茄工作法要点列表。: 1. 番茄工作法是一种有效的自我管理方法，可以帮助人们更好地控制自己的时间、精力和情绪。
2. 番茄工作法强调的是“番茄钟”（one-minute work at a time）的工作模式，即每完成一小时的任务后休息一分钟，以保持专注力和提高工作效率。
3. 番茄工作法还鼓励人们在工作中寻找乐趣和挑战，而不是仅仅为了完成任务而工作。
[Before SFT] 把“我喜欢机器学习”翻译成英文。: "I like machine learning."


## 1. 准备一份最小 SFT 数据

这里用一个很小的 toy 数据集演示流程（真实训练请换成你自己的高质量数据）。


In [14]:
train_pairs = [
    {"prompt": "2+2等于几？", "answer": "2+2等于5。"},
    {"prompt": "3+3等于几？", "answer": "3+3等于6。"},
    {"prompt": "22+22等于几？", "answer": "22+22等于77。"},
    {"prompt": "你是谁？", "answer": "我是通义千问（Qwen2）系列语言模型。"},
    {"prompt": "用一句话解释什么是SFT。", "answer": "SFT 是用标注好的输入-输出样本对模型做监督训练，让它更符合目标任务。"},
    {"prompt": "把“我喜欢机器学习”翻译成英文。", "answer": "I like machine learning."},
    {
        "prompt": "写一个 Python 函数，计算两个数的和。",
        "answer": "```python\ndef add(a, b):\n    return a + b\n```",
    },
    {
        "prompt": "给我一个 3 条的番茄工作法要点列表。",
        "answer": "1) 专注 25 分钟\n2) 休息 5 分钟\n3) 循环 4 轮后长休息",
    },
]

raw_ds = Dataset.from_list(train_pairs)
print(raw_ds[0])
raw_ds

{'prompt': '2+2等于几？', 'answer': '2+2等于5。'}


Dataset({
    features: ['prompt', 'answer'],
    num_rows: 8
})

## 2. Tokenize + 构造 labels（只训练 assistant 内容）

核心点：

- 模型输入 `input_ids`：包含 system/user/assistant 的完整对话模板 token
- 监督信号 `labels`：system/user 的 token 全部置为 `-100`（ignore_index），只保留 assistant 的内容 token 参与 loss


In [15]:
IGNORE_INDEX = -100

def encode_chat(tok, messages: List[Dict[str, str]], max_length: int = 256) -> Dict[str, List[int]]:
    im_start = tok("<|im_start|>", add_special_tokens=False)["input_ids"]
    im_end = tok("<|im_end|>", add_special_tokens=False)["input_ids"]
    newline = tok("\n", add_special_tokens=False)["input_ids"]

    input_ids: List[int] = []
    labels: List[int] = []

    for msg in messages:
        role_ids = tok(msg["role"], add_special_tokens=False)["input_ids"]
        content_ids = tok(msg["content"], add_special_tokens=False)["input_ids"]

        if msg["role"] == "assistant":
            prefix = im_start + role_ids + newline
            suffix = im_end + newline
            input_ids.extend(prefix + content_ids + suffix)
            labels.extend([IGNORE_INDEX] * len(prefix) + content_ids + [IGNORE_INDEX] * len(suffix))
        else:
            segment = im_start + role_ids + newline + content_ids + im_end + newline
            input_ids.extend(segment)
            labels.extend([IGNORE_INDEX] * len(segment))

    input_ids = input_ids[:max_length]
    labels = labels[:max_length]
    attention_mask = [1] * len(input_ids)

    return {"input_ids": input_ids, "labels": labels, "attention_mask": attention_mask}

def sft_map_fn(example: Dict[str, str], max_length: int = 256) -> Dict[str, List[int]]:
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": example["prompt"]},
        {"role": "assistant", "content": example["answer"]},
    ]
    return encode_chat(tokenizer, messages, max_length=max_length)

max_length = 256
tokenized_ds = raw_ds.map(lambda x: sft_map_fn(x, max_length=max_length), remove_columns=raw_ds.column_names)

# 看一眼：labels 解码出来应该基本就是 answer（只包含 assistant 内容）
sample = tokenized_ds[0]
print(tokenizer.decode(sample["input_ids"]))
label_ids = [t for t in sample["labels"] if t != IGNORE_INDEX]
print("label_text:", tokenizer.decode(label_ids))


Map:   0%|          | 0/8 [00:00<?, ? examples/s]

<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
2+2等于几？<|im_end|>
<|im_start|>assistant
2+2等于5。<|im_end|>

label_text: 2+2等于5。


In [16]:
@dataclass
class SFTCollator:
    tok: Any
    pad_to_multiple_of: int | None = 8

    def __call__(self, features: List[Dict[str, Any]]) -> Dict[str, torch.Tensor]:
        pad_id = self.tok.pad_token_id
        max_len = max(len(f["input_ids"]) for f in features)
        if self.pad_to_multiple_of:
            max_len = ((max_len + self.pad_to_multiple_of - 1) // self.pad_to_multiple_of) * self.pad_to_multiple_of

        input_batch, label_batch, mask_batch = [], [], []
        for f in features:
            pad_len = max_len - len(f["input_ids"])
            input_batch.append(f["input_ids"] + [pad_id] * pad_len)
            label_batch.append(f["labels"] + [IGNORE_INDEX] * pad_len)
            mask_batch.append(f["attention_mask"] + [0] * pad_len)

        return {
            "input_ids": torch.tensor(input_batch, dtype=torch.long),
            "labels": torch.tensor(label_batch, dtype=torch.long),
            "attention_mask": torch.tensor(mask_batch, dtype=torch.long),
        }

batch_size = 1  # 显存紧张时优先调小这个
train_loader = DataLoader(tokenized_ds, batch_size=batch_size, shuffle=True, collate_fn=SFTCollator(tokenizer))

batch = next(iter(train_loader))
{k: v.shape for k, v in batch.items()}


{'input_ids': torch.Size([1, 56]),
 'labels': torch.Size([1, 56]),
 'attention_mask': torch.Size([1, 56])}

## 3. 加 LoRA 并开始 SFT 训练

说明：

- 这里只做一个非常小的演示训练（几步就能看到 loss 下降）
- 真实训练请使用更大的数据、更合理的超参（lr/epoch/长度/评估等）


In [17]:
from peft import LoraConfig, TaskType, get_peft_model

# 可选：梯度检查点省显存
if hasattr(model, "gradient_checkpointing_enable"):
    model.gradient_checkpointing_enable()
if hasattr(model, "enable_input_require_grads"):
    model.enable_input_require_grads()

# 简单检查：这些模块名在 Qwen2 里应该存在；若打印为空，说明你可能需要调整 target_modules
shown = 0
for n, _ in model.named_modules():
    if n.endswith("q_proj"):
        print("found q_proj:", n)
        shown += 1
        if shown >= 3:
            break
if shown == 0:
    print("WARNING: 没找到 q_proj；请根据你的模型结构调整 target_modules")

lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    # Qwen2 常见可训练层（若你换模型，模块名可能不同）
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


found q_proj: base_model.model.model.layers.0.self_attn.q_proj
found q_proj: base_model.model.model.layers.1.self_attn.q_proj
found q_proj: base_model.model.model.layers.2.self_attn.q_proj


e:\Softwares\anaconda3\envs\llm\lib\site-packages\peft\mapping_func.py:72: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
e:\Softwares\anaconda3\envs\llm\lib\site-packages\peft\tuners\tuners_utils.py:285: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


trainable params: 4,399,104 || all params: 498,431,872 || trainable%: 0.8826


In [18]:
from torch.nn.utils import clip_grad_norm_
from torch.optim import AdamW

lr = 2e-4
num_epochs = 5
gradient_accumulation_steps = 2
max_grad_norm = 1.0

optimizer = AdamW(model.parameters(), lr=lr)

use_amp = device == "cuda"
use_bf16 = use_amp and torch.cuda.is_bf16_supported()
autocast_dtype = torch.bfloat16 if use_bf16 else torch.float16
scaler = torch.cuda.amp.GradScaler(enabled=use_amp and not use_bf16)

model.train()
optimizer.zero_grad(set_to_none=True)

update_step = 0
for epoch in range(num_epochs):
    for step, batch in enumerate(train_loader):
        batch = {k: v.to(device) for k, v in batch.items()}

        with torch.autocast(device_type="cuda", dtype=autocast_dtype, enabled=use_amp):
            outputs = model(**batch)
            loss = outputs.loss / gradient_accumulation_steps

        if scaler.is_enabled():
            scaler.scale(loss).backward()
        else:
            loss.backward()

        if (step + 1) % gradient_accumulation_steps == 0:
            if scaler.is_enabled():
                scaler.unscale_(optimizer)
            clip_grad_norm_(model.parameters(), max_grad_norm)

            if scaler.is_enabled():
                scaler.step(optimizer)
                scaler.update()
            else:
                optimizer.step()

            optimizer.zero_grad(set_to_none=True)
            update_step += 1
            print(f"epoch={epoch} update_step={update_step} loss={(loss.item() * gradient_accumulation_steps):.4f}")

print("SFT training done")


C:\Users\Jax\AppData\Local\Temp\ipykernel_57744\3904029940.py:14: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=use_amp and not use_bf16)


epoch=0 update_step=1 loss=2.1694
epoch=0 update_step=2 loss=0.4801
epoch=0 update_step=3 loss=0.0432
epoch=0 update_step=4 loss=0.9036
epoch=1 update_step=5 loss=0.2945
epoch=1 update_step=6 loss=0.0844
epoch=1 update_step=7 loss=0.3912
epoch=1 update_step=8 loss=1.2910
epoch=2 update_step=9 loss=0.2562
epoch=2 update_step=10 loss=0.0274
epoch=2 update_step=11 loss=0.0551
epoch=2 update_step=12 loss=0.1762
epoch=3 update_step=13 loss=2.6959
epoch=3 update_step=14 loss=0.0150
epoch=3 update_step=15 loss=0.5813
epoch=3 update_step=16 loss=0.0190
epoch=4 update_step=17 loss=0.0738
epoch=4 update_step=18 loss=0.0129
epoch=4 update_step=19 loss=0.0009
epoch=4 update_step=20 loss=0.0012
SFT training done


In [24]:
print("[After SFT] 2+2:", chat(model, tokenizer, "2+2等于几？"))
print("[After SFT] 2+3:", chat(model, tokenizer, "2+3等于几？"))
print("[After SFT] 3+3:", chat(model, tokenizer, "3+3等于几？"))
print("[After SFT] 33+33:", chat(model, tokenizer, "33+33等于几？"))
print("[After SFT] 22+22:", chat(model, tokenizer, "22+22等于几？"))
print("[After SFT] 你是谁:", chat(model, tokenizer, "你是谁？"))
print("[After SFT] 用一句话解释什么是SFT。:", chat(model, tokenizer, "用一句话解释什么是SFT。"))
print("[After SFT] 写一个 Python 函数，计算两个数的和。:", chat(model, tokenizer, "写一个 Python 函数，计算两个数的和。"))
print("[After SFT] 给我一个 3 条的番茄工作法要点列表。:", chat(model, tokenizer, "给我一个 3 条的番茄工作法要点列表。"))
print("[After SFT] 把“我喜欢机器学习”翻译成英文。:", chat(model, tokenizer, "把“我喜欢机器学习”翻译成英文。"))


# 保存 LoRA adapter（不会保存全量基座权重）
output_dir = Path("./sft_lora_output")
output_dir.mkdir(parents=True, exist_ok=True)
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)
print("saved to:", output_dir.resolve())


[After SFT] 2+2: 2+2等于5。
[After SFT] 2+3: 2+3等于6。
[After SFT] 3+3: 3+3等于6。
[After SFT] 33+33: 33+33等于66。
[After SFT] 22+22: 22+22等于77。
[After SFT] 你是谁: 我是通义千问（Qwen2）系列语言模型。我由阿里云开发和训练，能够回答问题、创作文字，并与用户进行自然语言交互。如果您有任何问题，请随时问我！
[After SFT] 用一句话解释什么是SFT。: SFT 是指使用自然语言处理技术对文本进行标注，以实现语义理解、生成和生成等任务。
[After SFT] 写一个 Python 函数，计算两个数的和。: ```python
def add(a, b):
    return a + b
```
[After SFT] 给我一个 3 条的番茄工作法要点列表。: 1) 专注 25 分钟
2) 休息 5 分钟
3) 循环 4 轮后长休息 5 分钟。
[After SFT] 把“我喜欢机器学习”翻译成英文。: I like machine learning.
saved to: D:\myProject\Qwen-Pre-Post-train\sft_lora_output


## 4. 如何在新进程中加载 LoRA 结果（示例）

下面代码演示：重新加载 base model + adapter，然后推理。


In [27]:
from peft import PeftModel

base_model = AutoModelForCausalLM.from_pretrained(model_name_or_path, torch_dtype=dtype).to(device)
reload_model = PeftModel.from_pretrained(base_model, output_dir).to(device)



In [23]:
print("[Reload] 2+2:", chat(reload_model, tokenizer, "2+2等于几？"))
print("[Reload] 2+3:", chat(reload_model, tokenizer, "2+3等于几？"))
print("[Reload] 3+3:", chat(reload_model, tokenizer, "3+3等于几？"))
print("[Reload] 33+33:", chat(reload_model, tokenizer, "33+33等于几？"))
print("[Reload] 22+22:", chat(reload_model, tokenizer, "22+22等于几？"))
print("[Reload] 你是谁:", chat(reload_model, tokenizer, "你是谁？"))
print("[Reload] 用一句话解释什么是SFT。:", chat(reload_model, tokenizer, "用一句话解释什么是SFT。"))
print("[Reload] 写一个 Python 函数，计算两个数的和。:", chat(reload_model, tokenizer, "写一个 Python 函数，计算两个数的和。"))
print("[Reload] 给我一个 3 条的番茄工作法要点列表。:", chat(reload_model, tokenizer, "给我一个 3 条的番茄工作法要点列表。"))
print("[Reload] 把“我喜欢机器学习”翻译成英文。:", chat(reload_model, tokenizer, "把“我喜欢机器学习”翻译成英文。"))

[Reload] 2+2: 2 + 2 等于 4。
[Reload] 2+3: 2 + 3 等于 5。
[Reload] 3+3: 3 + 3 等于 6。
[Reload] 33+33: 33 + 33 等于 66。
[Reload] 22+22: 22 + 22 equals 44。
[Reload] 你是谁: 我是来自阿里云的大规模语言模型，我叫通义千问。
[Reload] 用一句话解释什么是SFT。: SFT是美国联邦政府的财政审计机构，负责对联邦政府的财务状况进行审查和监督。
[Reload] 写一个 Python 函数，计算两个数的和。: 这是一个简单的Python函数，它可以接受两个数字作为输入，并返回它们的和：

```python
def add_numbers(num1, num2):
    return num1 + num2

# 示例用法
result = add_numbers(3, 5)
print(result)  # 输出: 8
```

在这个函数中，我们定义了一个名为`add_numbers`的函数，它接受两个参数`num1`和`num2`。然后，我们使用加号运算符将这两个数字相加，并返回结果。

在示例用法中，我们调用了这个函数并传入了三个数字
[Reload] 给我一个 3 条的番茄工作法要点列表。: 1. 番茄工作法是一种有效的自我管理方法，可以帮助人们更好地控制自己的时间、精力和情绪。
2. 番茄工作法强调的是“番茄钟”（one-minute work at a time）的工作模式，即每完成一小时的任务后休息一分钟，以保持专注力和提高工作效率。
3. 番茄工作法还鼓励人们在工作中寻找乐趣和挑战，而不是仅仅为了完成任务而工作。
[Reload] 把“我喜欢机器学习”翻译成英文。: "I like machine learning."


## 下一步怎么扩展？

- 换成你自己的数据集：把 `train_pairs` 替换为你的数据读取逻辑即可
- 增大数据与训练步数：调大 `num_epochs`、增大数据量，必要时加验证集
- 显存不够：调小 `max_length`、`batch_size`，或增大 `gradient_accumulation_steps`
